<div style="padding:16px;color:white;margin:0;font-size:180%;text-align:left;display:fill;border-radius:5px;background-color:#48C9B0 ;overflow:hidden"><center>Kaggle JPX Leaderboard Submission</center></div>

This notebook describes an example of training and building a model with simple feature generation to leaderboard.


# Introduction of Features from Source Data

The feature data will be stored at the following two directory: 

* Training Data:  `/train_files`

* Supplemental Data:  `/supplemental_files`

In [ ]:
import os
from decimal import ROUND_HALF_UP, Decimal

import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from tqdm import tqdm

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/jpx-tokyo-stock-exchange-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# set base_dir to load data
base_dir = "../input/jpx-tokyo-stock-exchange-prediction"
# There are three types of stock_price.csv
# We use one in the train_files folder for this notebook.
train_files_dir = f"{base_dir}/train_files"

# for forecasting phase leaderboard, you may want to include stock_price.csv in the supplemental_files folder.
# You can remove "forecasting phase leaderboard" comments in this notebook to use stock_price.csv in the supplemental_files folder.
# forecasting phase leaderboard:
support_files_dir = f"{base_dir}/supplemental_files"


In [ ]:
# load stock price data
df_price = pd.read_csv(f"{train_files_dir}/stock_prices.csv")

# forecasting phase leaderboard:
df_price_supplemental = pd.read_csv(f"{support_files_dir}/stock_prices.csv")
df_price = pd.concat([df_price, df_price_supplemental])


In [ ]:
df_price.tail()


In [ ]:
stock_list = pd.read_csv(os.path.join(base_dir,'stock_list.csv'))

# include Date
stock_list['Date'] = stock_list['EffectiveDate'].apply(lambda x: pd.to_datetime(x, format="%Y%m%d")\
                                                            .strftime("%Y-%m-%d")
                                                       )

share_df = pd.pivot_table(stock_list,
                           values='IssuedShares',
                           index= "SecuritiesCode", columns="Date",
                           aggfunc="first")

# trading share as of 2021
share_df.head()

# Pre-processing for model building

This notebook presents a simple model using LightGBM.

First, the features are generated using the price change and historical volatility described above.

In [ ]:
def get_features_for_predict(price, code):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    close_col = "AdjustedClose"
    feats = price.loc[price["SecuritiesCode"] == code, ["SecuritiesCode", close_col]].copy()

    # calculate 2 week return using AdjustedClose
    feats["return_2week"] = feats[close_col].pct_change(10)
    # calculate last 1 month return using AdjustedClose
    feats["return_1month"] = feats[close_col].pct_change(21)
    # calculate last 3 months return using AdjustedClose
    feats["return_3month"] = feats[close_col].pct_change(63)

    # calculate 2 week historical volatility using AdjustedClose
    feats["volatility_2week"] = (
        np.log(feats[close_col]).diff().rolling(10).std()
    )
    # calculate last 1 month historical volatility using AdjustedClose
    feats["volatility_1month"] = (
        np.log(feats[close_col]).diff().rolling(21).std()
    )
    # calculate last 3 months historical volatility using AdjustedClose
    feats["volatility_3month"] = (
        np.log(feats[close_col]).diff().rolling(63).std()
    )

    # filling data for nan and inf
    feats = feats.fillna(0)
    feats = feats.replace([np.inf, -np.inf], 0)
    # drop AdjustedClose column
    feats = feats.drop([close_col], axis=1)

    return feats

In [ ]:
# fetch prediction target SecuritiesCodes
codes = sorted(df_price["SecuritiesCode"].unique())
len(codes)

# Label creation

Next, we obtain the labels to be used for training the model (this is where we load and split the label data).

In [ ]:
def get_label(price, code):
    """ Labelizer
    Args:
        price (pd.DataFrame): dataframe of stock_price.csv
        code (int): Local Code in the universe
    Returns:
        df (pd.DataFrame): label data
    """
    df = price.loc[price["SecuritiesCode"] == code].copy()
    df.loc[:, "label"] = df["Target"]

    return df.loc[:, ["SecuritiesCode", "label"]]

In [ ]:
# split data into TRAIN and TEST
TRAIN_END = "2021-12-01"
# We put a week gap between TRAIN_END and TEST_START
# to avoid leakage of test data information from label
TEST_START = "2021-12-06"

def get_features_and_label(price, codes, features):
    """
    Args:
        price (pd.DataFrame): loaded price data
        codes  (array) : target codes
        feature (pd.DataFrame): features
    Returns:
        train_X (pd.DataFrame): training data
        train_y (pd.DataFrame): label for train_X
        test_X (pd.DataFrame): test data
        test_y (pd.DataFrame): label for test_X
    """
    # to store splited data
    trains_X, tests_X = [], []
    trains_y, tests_y = [], []

    # generate feature one by one
    for code in tqdm(codes):

        feats = features[features["SecuritiesCode"] == code].dropna()
        labels = get_label(price, code).dropna()

        if feats.shape[0] > 0 and labels.shape[0] > 0:
            # align label and feature indexes
            labels = labels.loc[labels.index.isin(feats.index)]
            feats = feats.loc[feats.index.isin(labels.index)]

            assert (labels.loc[:, "SecuritiesCode"] == feats.loc[:, "SecuritiesCode"]).all()
            labels = labels.loc[:, "label"]

            # split data into TRAIN and TEST
            _train_X = feats[: TRAIN_END]
            _test_X = feats[TEST_START:]

            _train_y = labels[: TRAIN_END]
            _test_y = labels[TEST_START:]
            
            assert len(_train_X) == len(_train_y)
            assert len(_test_X) == len(_test_y)

            # store features
            trains_X.append(_train_X)
            tests_X.append(_test_X)
            # store labels
            trains_y.append(_train_y)
            tests_y.append(_test_y)
            
    # combine features for each codes
    train_X = pd.concat(trains_X)
    test_X = pd.concat(tests_X)
    # combine label for each codes
    train_y = pd.concat(trains_y)
    test_y = pd.concat(tests_y)

    return train_X, train_y, test_X, test_y

In [ ]:
# generate feature/label
#train_X, train_y, test_X, test_y = get_features_and_label(
#    df_price, codes, feature
#)

# Building a simple model

Using the created features and labels, build a model using the following procedure

In [ ]:
lgbm_params = {
    'seed': 42,
    'n_jobs': -1,
}

feat_cols = [
    "return_2week",
    "return_1month",
    "return_3month",
    "volatility_2week",
    "volatility_1month",
    "volatility_3month",
]

In [ ]:
if False:
    # initialize model
    pred_model = LGBMRegressor(**lgbm_params)
    # train
    pred_model.fit(train_X[feat_cols].values, train_y)
    # prepare result data
    result = test_X[["SecuritiesCode"]].copy()
    # predict
    result.loc[:, "predict"] = pred_model.predict(test_X[feat_cols])
    # actual result
    result.loc[:, "Target"] = test_y.values

    def set_rank(df):
        """
        Args:
            df (pd.DataFrame): including predict column
        Returns:
            df (pd.DataFrame): df with Rank
        """
        # sort records to set Rank
        df = df.sort_values("predict", ascending=False)
        # set Rank starting from 0
        df.loc[:, "Rank"] = np.arange(len(df["predict"]))
        return df

    result = result.sort_values(["Date", "predict"], ascending=[True, False])
    result = result.groupby("Date").apply(set_rank)
    result.tail()

<div style="padding:16px;color:white;margin:0;font-size:180%;text-align:left;display:fill;border-radius:5px;background-color:#ABEBC6;overflow:hidden"><center>Evaluation Section</center></div>


# Performance Evaluation

Input the output of the forecasts of the constructed model into the evaluation function and plot the daily returns.

The evaluation function for this competition is as follows.

Please read [here](https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition) to know the evaluation function more.

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
# calc spread return sharpe
#calc_spread_return_sharpe(result, portfolio_size=200)

Then, we will show daily spread return of the model.

In [ ]:
def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): spread return
    """
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

#df_result = result.groupby('Date').apply(_calc_spread_return_per_day, 200, 2)

In [ ]:
#df_result.plot(figsize=(20, 8))

We also show a cumulative spread return of the mode

In [ ]:
# wealth curve
#df_result.cumsum().plot(figsize=(20, 8))

The model in this notebook is now complete! Try different features and training methods through trial and error!

# Saving model

You need to save your model parameter to use created model for your submission.

In [ ]:
#pred_model.booster_.save_model("simple-model.txt")

# Submission

In [ ]:
import sys
from decimal import ROUND_HALF_UP, Decimal

import numpy as np
import pandas as pd
from lightgbm import Booster, LGBMRegressor



In [ ]:
base_dir = "../input/jpx-tokyo-stock-exchange-prediction"

train_files_dir = f"{base_dir}/train_files"
supplemental_files_dir = f"{base_dir}/supplemental_files"

# model_file = "../input/simplemodel/simple-model.txt"
model_file = "./simple-model.txt"

In [ ]:
# feature column names
feat_cols = [
    "return_2week",
    "return_1month",
    "return_3month",
    "volatility_2week",
    "volatility_1month",
    "volatility_3month",
]

def generate_adjusted_close(df):
    """
    Args:
        df (pd.DataFrame)  : stock_price for a single SecuritiesCode
    Returns:
        df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
    """
    # sort data to generate CumulativeAdjustmentFactor
    df = df.sort_values("Date", ascending=False)
    # generate CumulativeAdjustmentFactor
    df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
    # generate AdjustedClose
    df.loc[:, "AdjustedClose"] = (
        df["CumulativeAdjustmentFactor"] * df["Close"]
    ).map(lambda x: float(
        Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
    ))
    # reverse order
    df = df.sort_values("Date")
    # to fill AdjustedClose, replace 0 into np.nan
    df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
    # forward fill AdjustedClose
    df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
    return df

def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # copy to edit
    price = price.copy()
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    price.set_index("Date", inplace=True)
    return price

def get_features_for_predict(price, code):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    close_col = "AdjustedClose"
    feats = price.loc[price["SecuritiesCode"] == code, ["SecuritiesCode", close_col]].copy()

    # calculate 2 week return using AdjustedClose
    feats["return_2week"] = feats[close_col].pct_change(10)
    # calculate last 1 month return using AdjustedClose
    feats["return_1month"] = feats[close_col].pct_change(21)
    # calculate last 3 months return using AdjustedClose
    feats["return_3month"] = feats[close_col].pct_change(63)

    # calculate 2 week historical volatility using AdjustedClose
    feats["volatility_2week"] = (
        np.log(feats[close_col]).diff().rolling(10).std()
    )
    # calculate last 1 month historical volatility using AdjustedClose
    feats["volatility_1month"] = (
        np.log(feats[close_col]).diff().rolling(21).std()
    )
    # calculate last 3 months historical volatility using AdjustedClose
    feats["volatility_3month"] = (
        np.log(feats[close_col]).diff().rolling(63).std()
    )

    # filling data for nan and inf
    feats = feats.fillna(0)
    feats = feats.replace([np.inf, -np.inf], 0)
    # drop AdjustedClose column
    feats = feats.drop([close_col], axis=1)

    return feats


### Live Mode
if True:
    # load stock price data
    df_price_raw = pd.read_csv(f"{train_files_dir}/stock_prices.csv")
    price_cols = [
        "Date",
        "SecuritiesCode",
        "Close",
        "AdjustmentFactor",
    ]
    df_price_raw = df_price_raw[price_cols]

    # filter data to reduce culculation cost 
    df_price_raw = df_price_raw.loc[df_price_raw["Date"] >= "2021-08-01"]

    # forecasting phase leaderboard:
    df_price_supplemental = pd.read_csv(f"{supplemental_files_dir}/stock_prices.csv")
    df_price_supplemental = df_price_supplemental[price_cols]
    df_price_raw = pd.concat([df_price_raw, df_price_supplemental])
    
    print(" > The training history data is loaded!")
    
#pred_model = Booster(model_file=model_file)


---

## Official API Submission

Call the API function from `jpx_tokyo_market_prediction`


In [ ]:
# load Time Series API
import jpx_tokyo_market_prediction
# make Time Series API environment (this function can be called only once in a session)
env = jpx_tokyo_market_prediction.make_env()
# get iterator to fetch data day by day
iter_test = env.iter_test()

In [ ]:
counter = 0
# fetch data day by day
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    current_date = prices["Date"].iloc[0]
    sample_prediction_date = sample_prediction["Date"].iloc[0]
    print(f" => Current_date: {current_date}, sample_prediction_date: @{sample_prediction_date}")

    if counter == 0:
        # to avoid data leakage
        df_price_raw = df_price_raw.loc[df_price_raw["Date"] < current_date]

    ################  Data Loading  ################
    # filter data to reduce culculation cost
    threshold = (pd.Timestamp(current_date) - pd.offsets.BDay(80)).strftime("%Y-%m-%d")
    print(f"threshold: {threshold}")
    df_price_raw = df_price_raw.loc[df_price_raw["Date"] >= threshold]

    # to generate AdjustedClose, increment price data
    df_price_raw = pd.concat([df_price_raw, prices[price_cols]])
    # generate AdjustedClose
    df_price = adjust_price(df_price_raw)

    # get target SecuritiesCodes
    codes = sorted(prices["SecuritiesCode"].unique())
    
    ################  Feature Processing  ################
    # generate feature
    #feature = pd.concat([get_features_for_predict(df_price, code) for code in codes])
    # filter feature for this iteration
    #feature = feature.loc[feature.index == current_date]
    feature = df_price.loc[current_date].copy()
    
    size = feature['AdjustedClose'].values  * share_df.loc[feature['SecuritiesCode'].values].iloc[:,0].astype(float)
    
    
    ################  Model Inference  ################
    
    # prediction
    feature.loc[:, "predict"] =  -np.log(size)  #pred_model.predict(feature[feat_cols])
    
    # set rank by predict
    feature = feature.sort_values("predict", ascending=False).drop_duplicates(subset=['SecuritiesCode'])
    feature.loc[:, "Rank"] = np.arange(len(feature))
    feature_map = feature.set_index('SecuritiesCode')['Rank'].to_dict()
    
    ##############  Prediction Uploading  ################
    
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)

    # check Rank
    assert sample_prediction["Rank"].notna().all()
    assert sample_prediction["Rank"].min() == 0
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1

    # register your predictions
    env.predict(sample_prediction)
    counter += 1

# Reference